In [ ]:
# Função de Rastrigin
def rastrigin(x):
    return 10*len(x) + sum([(xi**2 - 10 * np.cos(2 * np.pi * xi)) for xi in x])

class Particle:
    def __init__(self, bounds):
        self.position = np.random.uniform(bounds[:, 0], bounds[:, 1])
        self.velocity = np.random.uniform(-1, 1, size=(bounds.shape[0],))
        self.best_position = np.copy(self.position)
        self.best_score = rastrigin(self.position)

class PSO:
    def __init__(self, n_particles, bounds, iterations, c1=2.05, c2=2.05):
        self.n_particles = n_particles
        self.bounds = bounds
        self.iterations = iterations
        self.c1 = c1
        self.c2 = c2
        self.particles = [Particle(bounds) for _ in range(n_particles)]
        self.gbest_score = np.inf
        self.gbest_position = np.zeros(bounds.shape[0])
        self.w = np.linspace(0.9, 0.1, self.iterations)
        self.best_scores = []  # lista para armazenar o melhor score de cada iteração
        self.positions_over_time = []  # lista para armazenar as posições das partículas ao longo do tempo

    def optimize(self):
        for i in range(self.iterations):
            positions = []
            for particle in self.particles:
                fitness = rastrigin(particle.position)

                if fitness < particle.best_score:
                    particle.best_score = fitness
                    particle.best_position = np.copy(particle.position)

                if fitness < self.gbest_score:
                    self.gbest_score = fitness
                    self.gbest_position = np.copy(particle.position)

                r1 = np.random.random()
                r2 = np.random.random()

                cognitive = self.c1 * r1 * (particle.best_position - particle.position)
                social = self.c2 * r2 * (self.gbest_position - particle.position)

                particle.velocity = self.w[i] * particle.velocity + cognitive + social
                particle.position += particle.velocity

                particle.position = np.clip(particle.position, self.bounds[:, 0], self.bounds[:, 1])
                positions.append(particle.position)

            self.positions_over_time.append(positions)
            self.best_scores.append(self.gbest_score)

        return self.gbest_position, self.gbest_score

    def animate(self):
        fig, ax = plt.subplots()

        # Escolhendo aleatoriamente duas dimensões para plotar
        dim1, dim2 = np.random.choice(range(self.bounds.shape[0]), size=2, replace=False)

        def update(num):
            ax.clear()
            ax.set_title('Iteração ' + str(num))
            ax.set_xlim([self.bounds[dim1, 0], self.bounds[dim1, 1]])
            ax.set_ylim([self.bounds[dim2, 0], self.bounds[dim2, 1]])
            positions = self.positions_over_time[num]
            for position in positions:
                ax.scatter(position[dim1], position[dim2])

        ani = animation.FuncAnimation(fig, update, frames=self.iterations, repeat=False)
        #ani = animation.FuncAnimation(fig, animate, frames=self.iterations, interval=20, blit=True)
        writervideo = animation.FFMpegWriter(fps=2)
        ani.save('pso.mp4', writer=writervideo)
        plt.show()

def main():
    bounds = np.array([[-5.12, 5.12], [-5.12, 5.12], [-5.12, 5.12]]) # Limites do problema para a função de Rastrigin
    pso = PSO(n_particles=30, bounds=bounds, iterations=100)
    best_position, best_score = pso.optimize()
    print(f"Melhor posição : {best_position}, Melhor score : {best_score}")
    # pso.animate() # chamando a função de animação

main()
